In [2]:
import pathlib
import pandas as pd
import csv
import json
import zipfile
from PIL import Image
import torch
from torch.nn.functional import one_hot
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, Dataset    

In [34]:
class CustomDataset(Dataset):
    def __init__(self, path, label2id, id2label, transform=None) -> None:
        super().__init__()
        self.path = pathlib.Path(path)
        self.label2id = label2id
        self.id2label = id2label
        assert self.path.exists(), f"{path} does not exist"
        self.transform = transform
        self.files_path = self._get_file_list()
    
    def _get_file_list(self):
        files_path = []
            
        with zipfile.ZipFile(self.path, 'r') as zip_file:
            for name in zip_file.namelist():
                if '/' in name:
                    files_path.append(name)
        return files_path

    def __len__(self): 
        return len(self.files_path)
    
    def __getitem__(self, index):
        image, label = None, None
        with zipfile.ZipFile(self.path, 'r') as zip_file:
            with zip_file.open(self.files_path[index]) as file:
                image = Image.open(file).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
            label = one_hot(self.label2id[self.files_path[index].split('/')[1]])
        return image, label

In [35]:
def create_label_encs(dataset_path):
    label2id, id2label = {}, {}
    with zipfile.ZipFile(dataset_path, 'r') as zip_file:
        for name in zip_file.namelist():
            if '/' in name:
                label = name.split('/')[1]
                if label not in label2id:
                    label2id[label] = len(label2id)
                    id2label[len(id2label)] = label
    return label2id, id2label


label2id, id2label = create_label_encs(
    'datasets/geolocation-geoguessr-images-50k.zip')
print(label2id, id2label, sep='\n\n')

{'Aland': 0, 'Albania': 1, 'American Samoa': 2, 'Andorra': 3, 'Antarctica': 4, 'Argentina': 5, 'Armenia': 6, 'Australia': 7, 'Austria': 8, 'Bangladesh': 9, 'Belarus': 10, 'Belgium': 11, 'Bermuda': 12, 'Bhutan': 13, 'Bolivia': 14, 'Botswana': 15, 'Brazil': 16, 'Bulgaria': 17, 'Cambodia': 18, 'Canada': 19, 'Chile': 20, 'China': 21, 'Colombia': 22, 'Costa Rica': 23, 'Croatia': 24, 'Curacao': 25, 'Czechia': 26, 'Denmark': 27, 'Dominican Republic': 28, 'Ecuador': 29, 'Egypt': 30, 'Estonia': 31, 'Eswatini': 32, 'Faroe Islands': 33, 'Finland': 34, 'France': 35, 'Germany': 36, 'Ghana': 37, 'Gibraltar': 38, 'Greece': 39, 'Greenland': 40, 'Guam': 41, 'Guatemala': 42, 'Hong Kong': 43, 'Hungary': 44, 'Iceland': 45, 'India': 46, 'Indonesia': 47, 'Iraq': 48, 'Ireland': 49, 'Isle of Man': 50, 'Israel': 51, 'Italy': 52, 'Japan': 53, 'Jersey': 54, 'Jordan': 55, 'Kenya': 56, 'Kyrgyzstan': 57, 'Laos': 58, 'Latvia': 59, 'Lebanon': 60, 'Lesotho': 61, 'Lithuania': 62, 'Luxembourg': 63, 'Macao': 64, 'Madagas

In [36]:
dataset1 = CustomDataset('datasets/geolocation-geoguessr-images-50k.zip', label2id, id2label)
dataset2 = CustomDataset('datasets/geoguessr-55countries.zip', label2id, id2label)

49997 35977


In [ ]:
class CustomModel(nn.Module): #? Custom model made from scratch
    def __init__(self) -> None:
        super().__init__()
        

    def forward(self, x): pass

In [ ]:
class Callback:
    def __init__(self) -> None:
        self.epoch_counter = 0
        self.batch_counter = 0

    def on_train_begin(self): 
        self.batch_counter = 0

    def on_train_end(self):
        return None

    def on_epoch_begin(self): 
        return None
    
    def on_epoch_end(self, val_loss, model, *args): 
        self.epoch_counter += 1
    
    def on_batch_begin(self):
        return None
    
    def on_batch_end(self): 
        self.batch_counter += 1

In [ ]:
class ModelCheckpoint(Callback):
    def __init__(self, path) -> None:
        super().__init__()
        self.path = pathlib.Path(path)
        self.best_loss = float('inf')

    def on_epoch_end(self, val_loss, model, *args):
        super().on_epoch_end(val_loss, model, *args)
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            torch.save(model.state_dict(), self.path)

class EarlyStopping(Callback):
    def __init__(self, patience, restore_best_weights=False, checkpoint: str|bool=False) -> None:
        super().__init__()
        self.patience = patience
        self.restore_best_weights = restore_best_weights
        self.checkpoint = checkpoint

        if self.checkpoint:
            self.model_checkpoint = ModelCheckpoint(self.checkpoint)
        if self.restore_best_weights:
            self.best_weights = None

        self.counter = 0
        self.best_loss = float('inf')

    def on_epoch_end(self, val_loss, model, *args):
        super().on_epoch_end(val_loss, model, *args)
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping on epoch {self.epoch_counter} with val_loss: {val_loss}")
                return True
            
class LearningRateScheduler(Callback):
    def __init__(self, lr_scheduler) -> None:
        super().__init__()
        self.lr_scheduler = lr_scheduler

    def on_epoch_end(self, val_loss, model, *args):
        super().on_epoch_end(val_loss, model)
        self.lr_scheduler(self.epoch_counter, *args)

In [ ]:
def train(model, train_loader, val_loader, loss_func, optimizer, epochs, callbacks: None | list[Callback] = None):
    for epoch in range(epochs):
        if any(callback.on_epoch_begin() for callback in callbacks):
            break
        model.train()
        if any(callback.on_train_begin() for callback in callbacks):
            break
        for images, labels in train_loader:
            if any(callback.on_batch_begin() for callback in callbacks):
                break
            optimizer.zero_grad()
            output = model(images)
            loss = loss_func(output, labels)
            loss.backward()
            optimizer.step()
            if any(callback.on_batch_end() for callback in callbacks):
                break
        if any(callback.on_train_end() for callback in callbacks):
            break
        
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_samples = 0
            for images, labels in val_loader:
                output = model(images)
                loss = loss_func(output, labels)
                val_loss += loss.item()
                val_samples += 1
            avg_val_loss = val_loss / val_samples
            print(f"Epoch: {epoch}, Validation Loss: {avg_val_loss}")
        if any(callback.on_epoch_end(avg_val_loss, model) for callback in callbacks):
            break